# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [11]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import scipy
from scipy.io import arff
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag,ne_chunk
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import f1_score, classification_report,accuracy_score, scorer,make_scorer,fbeta_score

In [12]:
# load data from database
engine = create_engine('sqlite:///FigureEight.db')
df = pd.read_sql_table('FigureEight',engine)
X = df['message'].values
Y = df[df.columns[3:]].values
category_names = df.columns[3:]


### 2. Write a tokenization function to process your text data

In [13]:
def tokenize(text):
    '''
    input: 
        text : message text for processing
    output: 
        tokenized_word : message text after being processed 
    '''
    # Case Normalization
    lower_text = text.lower()
    # Punctutation removal
    ptext =re.sub(r"[^a-zA-Z0-9]"," ", lower_text)
    # Word Tokenization
    tokenword = word_tokenize(ptext)
    # stopword removal
    stopword = [w for w in tokenword if w not in stopwords.words('english')]
   # tokenized_word = []
    #for word in tokenword:
     #   tokenized_word.append(word if word not in stopwords.words('english')）
    lemmed_word=[]
    for word in stopword:
        lemmed_word.append(WordNetLemmatizer().lemmatize(word,pos='v'))
    #lemmed_word = [WordNetLemmatizer().lemmatize(word, pos = 'v') for word in stopword]
    return lemmed_word

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
pipeline = Pipeline([('vect',CountVectorizer(tokenizer = tokenize)),('tfidf',TfidfTransformer()),
                     ('clf',MultiOutputClassifier(OneVsRestClassifier(LinearSVC(random_state = 1))))])

### 4. Train pipeline
- Split data into train and test sets
- Train 


In [15]:
def train_pipeline(X,Y):
    '''
    input: X,Y used to train classifier
    output: fitted pipeline, train,test data

    '''
    X_train,X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2)
    pipeline.fit(X_train, Y_train)
    # train classifier
    return pipeline, X_train, X_test,Y_train,Y_test

   

In [16]:
model, X_train,X_test,Y_train,Y_test = train_pipeline(X,Y)

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
def evaluation (x_test, y_test):
    """
    input: testing dataset
    output: evaluation report
    """
   # model, X_train,X_test,Y_train,Y_test = train_pipeline(X,Y)
    Y_predict = model.predict(X_test)
    print(classification_report(Y_predict, Y_test, target_names = category_names))
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    for i in range(Y_test.shape[1]):
        print('%25s accuracy : %.2f' %(category_names[i], accuracy_score(Y_test[:,i], Y_predict[:,i])))
    
  

In [18]:
evaluation(X_test, Y_test)

                        precision    recall  f1-score   support

               related       0.91      0.86      0.88      4241
               request       0.58      0.74      0.65       687
                 offer       0.00      0.00      0.00         0
           aid_related       0.70      0.73      0.72      2052
          medical_help       0.30      0.57      0.39       200
      medical_products       0.28      0.68      0.40       101
     search_and_rescue       0.20      0.70      0.31        43
              security       0.01      0.11      0.02         9
              military       0.31      0.60      0.40        87
           child_alone       0.00      0.00      0.00         0
                 water       0.59      0.76      0.67       255
                  food       0.70      0.82      0.75       492
               shelter       0.56      0.79      0.66       322
              clothing       0.39      0.60      0.47        45
                 money       0.20      

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_f


### 6. Improve your model
Use grid search to find better parameters.

In [19]:
 parameters = {
            'tfidf__smooth_idf':[True, False],
            'clf__estimator__estimator__C': [1, 2, 5] 
                 }

cv = GridSearchCV(pipeline,param_grid=parameters,scoring= 'precision_samples', cv=3)
cv

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                            

In [20]:
cv.fit(X_train, Y_train)

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/Applications/anaconda3/lib/pyt

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                            

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
cv_predict = cv.predict(X_test)

In [25]:
print(classification_report(cv_predict,Y_test,target_names = category_names))

                        precision    recall  f1-score   support

               related       0.91      0.86      0.88      4241
               request       0.58      0.74      0.65       687
                 offer       0.00      0.00      0.00         0
           aid_related       0.70      0.73      0.72      2052
          medical_help       0.30      0.57      0.39       200
      medical_products       0.28      0.68      0.40       101
     search_and_rescue       0.20      0.70      0.31        43
              security       0.01      0.11      0.02         9
              military       0.31      0.60      0.40        87
           child_alone       0.00      0.00      0.00         0
                 water       0.59      0.76      0.67       255
                  food       0.70      0.82      0.75       492
               shelter       0.56      0.79      0.66       322
              clothing       0.39      0.60      0.47        45
                 money       0.20      

In [26]:
for i in range(Y_test.shape[1]):
          print('%25s accuracy : %.2f' %(category_names[i], accuracy_score(Y_test[:,i], cv_predict[:,i])))

                  related accuracy : 0.82
                  request accuracy : 0.89
                    offer accuracy : 1.00
              aid_related accuracy : 0.77
             medical_help accuracy : 0.93
         medical_products accuracy : 0.96
        search_and_rescue accuracy : 0.97
                 security accuracy : 0.98
                 military accuracy : 0.97
              child_alone accuracy : 1.00
                    water accuracy : 0.96
                     food accuracy : 0.95
                  shelter accuracy : 0.95
                 clothing accuracy : 0.99
                    money accuracy : 0.98
           missing_people accuracy : 0.99
                 refugees accuracy : 0.97
                    death accuracy : 0.97
                other_aid accuracy : 0.86
   infrastructure_related accuracy : 0.93
                transport accuracy : 0.96
                buildings accuracy : 0.96
              electricity accuracy : 0.98
                    tools accuracy

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [27]:
pickle.dump(model,open("pipeline.p","wb"))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.